In [1]:
# !pip install pytelegrambotapi

In [2]:
# !pip install annoy

In [3]:
# -- coding: utf-8 -

In [4]:
import telebot
import re
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [5]:
bot = telebot.TeleBot('1373860493:AAGpTiPZHSw0Sty5FStfZ6zUB12eTbnRVYo')

In [6]:
modelW2V = Word2Vec.load('modelw2v.bin')

In [7]:
w2v_index = annoy.AnnoyIndex(200 ,'angular')
w2v_index.load('w2v_index.bin')

True

In [8]:
index_map = pickle.load(open('index_map.pkl', 'rb'))

In [9]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

def preprocess_txt(line):   
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]    
    return spls

def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(200)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i] for i in answers]

In [10]:
@bot.message_handler(commands=['start'])
def startCommand(message):
    bot.send_message(message.chat.id, 'Привет, ты написал мне /start')

In [11]:
@bot.message_handler(content_types=['text'])
def textMessage(message):
    TEXT = message.text.lower()
    response = get_response(TEXT, w2v_index, modelW2V, index_map)
    bot.send_message(message.chat.id, response)

In [12]:
bot.polling()

C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  app.launch_new_instance()
C:\Users\marchenko-i-u\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
